In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.misc import comb
from sklearn.model_selection import train_test_split , KFold, StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import pdb
from sklearn.metrics import roc_auc_score , make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
#import NuestroArbol as ourTree
import random
from scipy.stats import randint as sp_randint
from time import time
import collections
from sklearn import preprocessing
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC


%matplotlib inline

# Importo datos 

In [2]:
X_comp = pd.read_csv('X_competencia.csv')
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
X.drop(['index'],inplace=True, axis=1)
y.drop(['index'], inplace=True,axis=1)



In [3]:
#partimos los datos en desarrollo(87%) y holdout(13%). 
X_desarrollo , X_holdout ,y_desarrollo, y_holdout = train_test_split(X, y['output'],
                                                                    test_size=0.13,random_state=0,stratify=y['output'])
#pase los kfold aca arriba ya que se usan en varios lugares.
#evaluar si esta bueno que siempre se usen los mismos folds
kfold = StratifiedKFold(n_splits=5)
kfold.get_n_splits(X_desarrollo,y_desarrollo)


5

In [4]:

lsvc = LinearSVC(C=0.01, penalty="l2", dual=False).fit(X_desarrollo, y_desarrollo)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X_desarrollo)
X_new.shape




(435, 19)

In [5]:
# Build a classification task using 3 informative features






# Create the RFE object and compute a cross-validated score.
svc = SVC(kernel="linear")
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=svc, step=60, cv=kfold,
              scoring='roc_auc')
rfecv.fit(X_desarrollo, y_desarrollo)
rfecv.transform(X_desarrollo).shape

(435, 20)

### K-fold CV 

In [6]:
def primerTablaEjercicio2(treeClasifier):
    
    accuracy_train=[]
    accuracy_valildation=[]
    ROC_AUC_train=[]
    ROC_AUC_validation=[]

    #este for itera sobre los k folds en cada loop tego un set de datos y otro de validacion
    for train, test  in kfold.split(X_desarrollo,y_desarrollo):
        #print("TRAIN:", train_index,'\n', "TEST:", test_index,'\n' )
        X_train, X_val = X_desarrollo.iloc[train], X_desarrollo.iloc[test]
        y_train, y_val = y_desarrollo.iloc[train], y_desarrollo.iloc[test]
        #intancio el arbol que voy a entrenar en cada fold
        tree = treeClasifier(max_depth=3, criterion="gini")

        tree.fit(X_train, y_train.astype(int))
        accuracy_train.append(tree.score(X=X_train, y=y_train))
        accuracy_valildation.append(tree.score(X=X_val, y=y_val))
        ROC_AUC_train.append(roc_auc_score(y_train,tree.predict(X_train)))
        ROC_AUC_validation.append(roc_auc_score(y_val,tree.predict(X_val)))
        
    return pd.DataFrame({ 'Partición' : np.arange(1,6),'Accuracy (training)' :accuracy_train,
                          'Accuracy (validación)' : accuracy_valildation,
                          'ROC AUC (training)' : ROC_AUC_train,
                          'ROC AUC (validación)' : ROC_AUC_validation})






# Tabla de precision

In [7]:
display(primerTablaEjercicio2(DecisionTreeClassifier))  

,Partición,Accuracy (training),Accuracy (validación),ROC AUC (training),ROC AUC (validación)
0,1,0.783862,0.659091,0.785980,0.660417
1,2,0.813218,0.758621,0.801587,0.752394
2,3,0.830460,0.597701,0.820954,0.588564
3,4,0.844828,0.597701,0.841170,0.592287
4,5,0.828080,0.639535,0.823049,0.630933


# Arboles combinaciones

In [8]:
def accuracyForTrainingAndValidation(depth,criteria,treeClasifier):
    acc_train=[]
    acc_val=[]


    #este for itera sobre los k folds en cada loop tego un set de training y otro de validacion
    for train, test  in kfold.split(X_desarrollo,y_desarrollo):
        #print("TRAIN:", train_index,'\n', "TEST:", test_index,'\n' )
        X_train, X_val = X_desarrollo.iloc[train], X_desarrollo.iloc[test]
        y_train, y_val = y_desarrollo.iloc[train], y_desarrollo.iloc[test]
        #intancio el arbol que voy a entrenar en cada fold
        tree = treeClasifier(max_depth=depth, criterion=criteria)

        tree.fit(X_train, y_train.astype(int))
        acc_train.append(tree.score(X=X_train, y=y_train))
        acc_val.append(tree.score(X=X_val, y=y_val))
        
    return {"training":np.mean(acc_train), "validation": np.mean(acc_val)}

        

In [9]:
def segundaTablaEjercicio2(treeClasifier):

    trainingResults=[]
    validationResults=[]
    evaluatedDepths=[]
    evaluatedCriterias=[]
    depthsDictionary={3:'3',5:'5',None:'Infinito'}
    depths = [3,5,None]    
    criterias =['gini','entropy']

    for depth in depths:
        for criteria in criterias:
            trainingResults.append(accuracyForTrainingAndValidation(depth,criteria,treeClasifier)["training"])
            validationResults.append(accuracyForTrainingAndValidation(depth,criteria,treeClasifier)["validation"])
            evaluatedDepths.append(depthsDictionary[depth])
            evaluatedCriterias.append(criteria)
    
    return pd.DataFrame({ 'Altura Máxima' : evaluatedDepths,
                          'Criterio de evaluación de corte' : evaluatedCriterias,
                          'Accuracy (training)' : trainingResults,
                          'Accuracy (validación)' : validationResults}).sort_values(by=['Criterio de evaluación de corte'],ascending=False)

# Tabla con combinaciones

In [10]:
display(segundaTablaEjercicio2(DecisionTreeClassifier))  

,Altura Máxima,Criterio de evaluación de corte,Accuracy (training),Accuracy (validación)
0,3,gini,0.819515,0.650530
2,5,gini,0.927579,0.673467
4,Infinito,gini,1.000000,0.661761
1,3,entropy,0.791947,0.682955
3,5,entropy,0.917243,0.694580
5,Infinito,entropy,1.000000,0.671483


# Ejercicio Extra: Resultados para nuestro clasificador

In [11]:
#display(primerTablaEjercicio2(ourTree.MiClasificadorArbol))  

In [12]:
#display(segundaTablaEjercicio2(ourTree.MiClasificadorArbol))
#esta tabla tardo 10 minutos en crearse, ver como mejorar performance

In [13]:
roc_auc_score = make_scorer(roc_auc_score)
n_iter_search = 20
scaler = preprocessing.StandardScaler().fit(X_desarrollo)
max_abs_scaler = preprocessing.MaxAbsScaler()                                      
def performGridSearch(clasiffier, param_grid):
    start = time()
    gridSearch = GridSearchCV(clasiffier, param_grid, cv=kfold, scoring=roc_auc_score)
    gridSearch.fit(X_new, y_desarrollo)
    return {"bestScore" : gridSearch.best_score_, "bestParams" : gridSearch.best_params_, "executionTime" : time() - start}

def performRandomSearch(classifier, param_dist):
    start = time()
    randomSearch = RandomizedSearchCV(classifier, param_distributions=param_dist, n_iter=n_iter_search, cv=kfold, scoring=roc_auc_score, refit=True)
    randomSearch.fit(X_new, y_desarrollo)
    return {"bestScore" : randomSearch.best_score_, "bestParams" : randomSearch.best_params_, "executionTime" : time() - start}


def displayBestParamsTable(classifier, paramsGridSearch, paramsRandomSearch):
    gridSearchResult = performGridSearch(classifier, paramsGridSearch);
    randomSearchResult = performRandomSearch(classifier, paramsRandomSearch);
    df =  pd.DataFrame({ ' ' : ["Mejor performance (Roc Auc)", "Tiempo de ejecución (Segundos)"],
                          'Grid Search' : [gridSearchResult["bestScore"],gridSearchResult[ "executionTime"]],
                          'Random Search' : [randomSearchResult["bestScore"],randomSearchResult[ "executionTime"]]
                        })      
    display(df.set_index(' '))
    orderedBestGridParams = collections.OrderedDict(sorted(gridSearchResult["bestParams"].items()))
    orderedBestRandomParams = collections.OrderedDict(sorted(randomSearchResult["bestParams"].items()))
    display(pd.DataFrame({ 'Mejores parámetros Grid Search' : list(orderedBestGridParams.keys()),
                          '' : list(orderedBestGridParams.values())
                        }).set_index('Mejores parámetros Grid Search'))
    display(pd.DataFrame({ 'Mejores parámetros Random Search' : list(orderedBestRandomParams.keys()),
                          '' : list(orderedBestRandomParams.values())
                        }).set_index('Mejores parámetros Random Search'))

# LDA

In [14]:
lda_param_grid = [{'solver': ['lsqr', 'eigen'], 'shrinkage': [None, 'auto', 0, 0.25, 0.5, 0.75, 1]},
                   {'solver': ['svd'], 'store_covariance' : [True, False]}]

lda_param_random = {'solver': ['lsqr', 'eigen'], 'shrinkage': np.arange(0, 1, 0.0001)}

displayBestParamsTable(LinearDiscriminantAnalysis(),lda_param_grid, lda_param_random)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:294: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  self.coef_ = linalg.lstsq(self.covariance_, self.means_.T)[0].T


,Grid Search,Random Search
,,
Mejor performance (Roc Auc),0.774381,0.771116
Tiempo de ejecución (Segundos),0.414439,0.591013


,
Mejores parámetros Grid Search,
shrinkage,None
solver,lsqr


,
Mejores parámetros Random Search,
shrinkage,0.3007
solver,lsqr


# Arbol de Decisión

In [15]:
tree_param_grid = [{'max_depth': [3,5,10,15,20],'criterion': ('gini','entropy'), 'splitter' : ['random', 'best'] }]

tree_param_random = {'max_depth': np.arange(3, 21),'criterion': ('gini','entropy'), 'splitter' : ['random', 'best'] }

displayBestParamsTable(DecisionTreeClassifier(),tree_param_grid, tree_param_random)



,Grid Search,Random Search
,,
Mejor performance (Roc Auc),0.738894,0.718736
Tiempo de ejecución (Segundos),0.749238,0.744391


,
Mejores parámetros Grid Search,
criterion,gini
max_depth,5
splitter,best


,
Mejores parámetros Random Search,
criterion,entropy
max_depth,19
splitter,best


# KNN

In [16]:
KNN_param_grid = {'n_neighbors': [5, 30, 50, 100],'p': np.arange(1,3), "weights" : ['uniform', 'distance']}
KNN_param_random = {'n_neighbors': np.arange(5, 101), 'p': np.arange(1,3), "weights" : ['uniform', 'distance']}
displayBestParamsTable(KNeighborsClassifier(),KNN_param_grid, KNN_param_random)

,Grid Search,Random Search
,,
Mejor performance (Roc Auc),0.770816,0.781108
Tiempo de ejecución (Segundos),1.274187,1.659559


,
Mejores parámetros Grid Search,
n_neighbors,30
p,2
weights,uniform


,
Mejores parámetros Random Search,
n_neighbors,23
p,2
weights,uniform


# SVM

In [17]:


SVM_param_grid = [{'kernel': ['rbf', 'poly', 'sigmoid'], 'gamma': [1e-1, 1e-5],
                     'C': [.0001, .1, 1, 100]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
SVM_param_random = {'kernel': ['rbf', 'poly', 'sigmoid'], 'gamma':np.arange(1e-5, 1e-1, 0.0001),
                     'C': np.arange(0.001,101)}

displayBestParamsTable(SVC(),SVM_param_grid, SVM_param_random)



,Grid Search,Random Search
,,
Mejor performance (Roc Auc),0.771832,0.785430
Tiempo de ejecución (Segundos),30.669055,1.131408


,
Mejores parámetros Grid Search,
C,1
kernel,linear


,
Mejores parámetros Random Search,
C,33.001
gamma,0.00741
kernel,sigmoid


In [18]:

#SVM_param_grid = [{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
 #                    'C': np.arange(0.001, 1, 0.01),'multi_class' :['ovr', 'multinomial', 'auto']}]
#SVM_param_random = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
  #                   'C': np.arange(0.001, 1, 0.01),'multi_class':['ovr', 'multinomial', 'auto']}

#logreg = LogisticRegression().fit(X_train, y_train)
#displayBestParamsTable(LogisticRegression(),SVM_param_grid, SVM_param_random)




In [19]:

#min_max_scaler = preprocessing.MinMaxScaler()
#>>> X_train_minmax = min_max_scaler.fit_transform(X_train)
#>>> X_train_minmax

In [20]:
#X_scaled = preprocessing.scale(X_desarrollo)